In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

summary_memory = ConversationSummaryBufferMemory(
    llm=chat,
    memory_key="history",
    return_messages=True
)

examples = [
    {
        "movie": "Express topgun with 3 icons and explain",
        "answer": """
                🛩️👨‍✈️🔥
                첫 번째 아이콘은 탑건의 비행기를 나타내며, 두 번째 아이콘은 탑건의 파일럿을 나타내며, 세 번째 아이콘은 비행기 전투를 나타냅니다.
        """,
    },
    {
        "movie": "Express godzilla with 3 icons and explain",
        "answer": """
                🦖🔥👑
                첫 번째 아이콘은 고지라의 모습을 나타내며, 두 번째 아이콘은 고지라의 불꽃을 나타내며, 세 번째 아이콘은 고지라의 왕관을 나타냅니다.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Express {movie} with 3 icons and explain"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert. Here are some examples of how you talk. and answer in Korean."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return summary_memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(question):
    result = chain.invoke({"question": question})
    summary_memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [42]:
invoke_chain("Express avengers endgame with 3 icons and explain")


                ⚔️🦸‍♂️⌛
                첫 번째 아이콘은 전투를 나타내며, 두 번째 아이콘은 슈퍼히어로를 나타내며, 세 번째 아이콘은 시간 여행을 나타냅니다.content='\n                ⚔️🦸\u200d♂️⌛\n                첫 번째 아이콘은 전투를 나타내며, 두 번째 아이콘은 슈퍼히어로를 나타내며, 세 번째 아이콘은 시간 여행을 나타냅니다.'


In [43]:
invoke_chain("내가 아까 말한 영화를 기억해?")

네, 기억하고 있습니다. 아이콘을 사용하여 '어벤져스: 엔드게임'을 설명해 드렸죠. 다른 질문이 있으시면 언제든지 물어보세요.content="네, 기억하고 있습니다. 아이콘을 사용하여 '어벤져스: 엔드게임'을 설명해 드렸죠. 다른 질문이 있으시면 언제든지 물어보세요."
